#### General Steps to Follow

1. Importing Packages
2. Loading the model
3. Collecting verification images
4. Preprocessing the input image
5. Defining a verification function
6. Getting the input and verify

## 1) Importing Packages

In [7]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Layer

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 2) Loading the model

#### The distance layer

In [8]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super(L1Dist, self).__init__(**kwargs)

    def call(self, anchor, validation):
        return tf.abs(anchor - validation)

In [9]:
siamese_model = tf.keras.models.load_model('../my data/siamese_model.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 3) Collecting verification images

#### Setup Paths

In [10]:
inp_img_path = "../my data/input_image" # Path for the input image that will be used in verification
ver_path = "../my data/verification"   # Path for some anchor images will be used to verify the input images

#### Collecting the verification images

In [11]:
ver_images = []
for img_name in os.listdir(ver_path):
    path = ver_path + "/" + img_name
    image = plt.imread(path)
    ver_images.append(image)
ver_images = np.array(ver_images)

print("Shape of verification images is:", ver_images.shape)

Shape of verification images is: (73, 250, 250, 3)


#### Normalizing the verification images

In [12]:
ver_images = ver_images/ [255]

#### Resizing the verification images to (105,105,3)

In [13]:
temp = []
for i in range(ver_images.shape[0]):
    img = ver_images[i]
    img = tf.image.resize(img,(105,105))
    img = img.numpy()
    temp.append(img)
temp = np.array(temp)
temp = temp.reshape((73,1,105,105,3))
ver_images = temp

print("Shape of the resized verification images is:", ver_images.shape)

Shape of the resized verification images is: (73, 1, 105, 105, 3)


### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 4) Preprocessing the input image

In [14]:
def image_transformation(img):
    # Normalizing the image
    img = img/255
    # resizing the image to (105, 105, 3)
    img = tf.image.resize(img,(105,105))
    img = img.numpy()
    img = img.reshape((1,105,105,3))
    
    return img

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 5) Defining a verification function

In [15]:
def verify(input_image):
    print("\nNumber of images will be used in verification is:", ver_images.shape[0])
    print("\n")
    
    result = []
    i = 1
    for ver in ver_images:
        print("Verification Image #", i); i+=1
        y_hat = siamese_model.predict([ver,input_image])
        if(y_hat[0,0] >= 0.5):
            y_hat[0,0] = 1
        else:
            y_hat[0,0] = 0
        result.append(y_hat[0,0])
    
    m = ver_images.shape[0]
    result = np.array(result)
    accuracy = 100*(np.sum(result == 1)/m)
    
    print("\nOutput:")
    print("Accuracy =", accuracy)
    if(accuracy > 60):
        print("Verified, Welcome Amro!")
    else:
        print("Not Recognized")

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

## 6) Getting the input and verify

In [16]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    input_image = frame.copy()
    
    # Show image back to screen
    cv2.imshow('Verification Image', frame)
    
    # Normalizing and resizing the image
    input_image = image_transformation(input_image)

    
    if cv2.waitKey(1) & 0XFF == ord('a'): 
        imgname = inp_img_path + "/" + "input_image.jpg"

        # Write out input image
        cv2.imwrite(imgname, frame)
                
        # Verification
        verify(input_image)
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()


Number of images will be used in verification is: 73


Verification Image # 1
1/1 [==============================] - 1s 814ms/step
Verification Image # 2
1/1 [==============================] - 0s 190ms/step
Verification Image # 3
1/1 [==============================] - 0s 216ms/step
Verification Image # 4
1/1 [==============================] - 0s 157ms/step
Verification Image # 5
1/1 [==============================] - 0s 174ms/step
Verification Image # 6
1/1 [==============================] - 0s 199ms/step
Verification Image # 7
1/1 [==============================] - 0s 156ms/step
Verification Image # 8
1/1 [==============================] - 0s 171ms/step
Verification Image # 9
1/1 [==============================] - 0s 191ms/step
Verification Image # 10
1/1 [==============================] - 0s 167ms/step
Verification Image # 11
1/1 [==============================] - 0s 199ms/step
Verification Image # 12
1/1 [==============================] - 0s 159ms/step
Verification Image # 13
1/1 [